<a href="https://colab.research.google.com/github/m10k1/ml-learn/blob/main/PaliGemma%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E3%81%BF%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')


必要なライブラリを読み込みます。
* torch
* immutabledict
* sentencepiece



In [2]:
!pip install kagglehub --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: kagglehub
    Found existing installation: kagglehub 0.3.5
    Uninstalling kagglehub-0.3.5:
      Successfully uninstalled kagglehub-0.3.5


In [3]:
!pip install -q -U immutabledict sentencepiece

In [4]:
!git clone https://github.com/google/gemma_pytorch.git

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 239 (delta 86), reused 53 (delta 53), pack-reused 121 (from 2)
Receiving objects: 100% (239/239), 2.18 MiB | 17.16 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [5]:
!mkdir /content/gemma/
!mv /content/gemma_pytorch/gemma/* /content/gemma/


In [10]:
import sys
import shutil
sys.path.append("/content/gemma_pytorch/")
from gemma.config import GemmaConfig, get_model_config
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
import contextlib
import os
import torch
import kagglehub

In [12]:
# Load the model
VARIANT = "2b-v2"
MACHINE_TYPE = "cuda"
model_path = kagglehub.model_download('google/gemma-2-2b-jpn-it/pyTorch/gemma-2-2b-jpn-it')

dest_dir = "/content/gemma/gemma-2-2b-jpn-it/pytorch/gemma-2-2b-jpn-it/1/"

if not os.path.exists(dest_dir):
  os.makedirs(dest_dir)

model_weights_path = os.path.join(dest_dir, "model.ckpt")
shutil.copy(os.path.join(model_path, "model.ckpt"), model_weights_path)

model_tokenizer_path = os.path.join(dest_dir, "tokenizer.model")
shutil.copy(os.path.join(model_path, "tokenizer.model"), model_tokenizer_path)


'/content/gemma/gemma-2-2b-jpn-it/pytorch/gemma-2-2b-jpn-it/1/tokenizer.model'

In [13]:
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

model_config = get_model_config(VARIANT)
model_config.tokenizer = model_tokenizer_path

device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  model.load_weights(model_weights_path)
  model = model.to(device).eval()

In [14]:
# Use the model
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n"

prompt = (
  USER_CHAT_TEMPLATE.format(
    prompt="「おにぎり」をテーマに短編を書いてください"
  )
  + "<start_of_turn>model\n"
)
print(prompt)

result = model.generate(
  prompt,
  device=device,
  output_len=256,
)
print(result)

<start_of_turn>user
「おにぎり」をテーマに短編を書いてください<end_of_turn>
<start_of_turn>model

## おにぎりの物語

春風に誘われるように、お寺の門前に佇む小さな店。その暖簾には、息づく「おにぎり」への想いが込める。

「あきら」は、長い時間をかけて培ってきた技。手際の良い握り、鮭の甘味、ご飯の炊き具合。すべて、家族の温もり、故郷の記憶、そして日常を忘れさせる、優しい味へのこだわりで紡がれていく。

ある日、その店に訪れたのは、疲れた心を持つ青年。「元気になりましょう」と店主は言った。青年の心が閉ざされていた。それは、故郷の忘れ去られた夢、そして将来への不安だった。

「おにぎり」は単なる食物ではなく、その形、その温度、その香りは、人生のあらゆる局面に寄り添う。

店主は、青年の前に「人生は、ただ一つのおにぎりと同じように、小さな形でも、温かい味でいっぱいのものです。」と言った。

青年は、初めて深く「おにぎり」を味わった。温まった匂いと、ご飯の香ばしさが、彼の人生を揺さぶった。

その日から、青年は再び、自身の夢に向かって歩き


パラメータをダウンロードします。